In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
plt.style.use('ggplot')

# Import Data

In [2]:
teams = pd.read_csv('data/MDataFiles_Stage1/MTeams.csv')
teams.info()
teams[teams['TeamName'].str.contains('Villanova|North Carolina')] # these teams actually never met in NCAA Tournament 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 4 columns):
TeamID           367 non-null int64
TeamName         367 non-null object
FirstD1Season    367 non-null int64
LastD1Season     367 non-null int64
dtypes: int64(3), object(1)
memory usage: 11.5+ KB


,TeamID,TeamName,FirstD1Season,LastD1Season
213,1314,North Carolina,1985,2020
336,1437,Villanova,1985,2020


In [3]:
seeds = pd.read_csv('data/MDataFiles_Stage1/MNCAATourneySeeds.csv')
seeds.info()
seeds.query("TeamID == 1314") # UNC seedings through the years 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2286 entries, 0 to 2285
Data columns (total 3 columns):
Season    2286 non-null int64
Seed      2286 non-null object
TeamID    2286 non-null int64
dtypes: int64(2), object(1)
memory usage: 53.7+ KB


,Season,Seed,TeamID
49,1985,Z02,1314
114,1986,Z03,1314
128,1987,W01,1314
241,1988,Z02,1314
305,1989,Z02,1314
343,1990,X08,1314
384,1991,W01,1314
499,1992,Z04,1314
512,1993,W01,1314
576,1994,W01,1314


In [4]:
tourney_results = pd.read_csv('data/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
tourney_results.info()
tourney_results.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2251 entries, 0 to 2250
Data columns (total 8 columns):
Season     2251 non-null int64
DayNum     2251 non-null int64
WTeamID    2251 non-null int64
WScore     2251 non-null int64
LTeamID    2251 non-null int64
LScore     2251 non-null int64
WLoc       2251 non-null object
NumOT      2251 non-null int64
dtypes: int64(7), object(1)
memory usage: 140.8+ KB


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [5]:
regular_results = pd.read_csv('data/MDataFiles_Stage1/MRegularSeasonCompactResults.csv') 
regular_results.info()
regular_results.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161552 entries, 0 to 161551
Data columns (total 8 columns):
Season     161552 non-null int64
DayNum     161552 non-null int64
WTeamID    161552 non-null int64
WScore     161552 non-null int64
LTeamID    161552 non-null int64
LScore     161552 non-null int64
WLoc       161552 non-null object
NumOT      161552 non-null int64
dtypes: int64(7), object(1)
memory usage: 9.9+ MB


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


# Helper Functions

In [6]:
def clf_eda_numeric(data, feature, target='Won'):
    """ Creates EDA plots for a binary target vs. a select numeric feature """
    
    # set up axes and define attributes 
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14,8))
    neutral_color = list(plt.rcParams['axes.prop_cycle'])[3]['color']
    
    # top-left: boxplot 
    sns.boxplot(x=target, y=feature, data=data, ax=axes[0,0]) 
    
    # top-right: overlapping density plots, 1 for each class  
    for y in [0,1]:
        subset = data[data[target]==y]
        sns.distplot(subset[feature], label=y, ax=axes[0,1])
        xmin, xmax = data[feature].min(), data[feature].max()
        axes[0,1].set_xlim(xmin, xmax)
        axes[0,1].set_ylabel('Density')
        axes[0,1].legend()
        
    # bottom-left: line plot of average target by feature value  
    sns.lineplot(x=feature, y=target, color=neutral_color, data=data, ax=axes[1,0])
    
    # bottom-right: histogram of combined data 
    sns.distplot(data[feature], kde=False, norm_hist=True, label='Density', color=neutral_color, ax=axes[1,1])

# Construct Dataset + EDA

### Create IDs and Labels

Each observation in our dataset is a March Madness match-up between two teams in a particular season, uniquely identified by a SSSS_XXXX_YYYY ID, where SSSS is the four digit season number, XXXX is the four-digit TeamID of the lower-ID team, and YYYY is the four-digit TeamID of the higher-ID team. The label of this observation is the outcome of the game, indicating whether the lower-ID team wins (1) or loses (0). 

In [23]:
def create_match_ups(seeds, seasons=None): 
    """ Given seeds dataframe and a tuple of (start, end) seasons, generate all possible match ups for each season """
    
    seeds = seeds.copy() 
    
    # filter for relevant seasons 
    if seasons is not None: 
        start_season, end_season = seasons 
        seeds = seeds[(seeds["Season"] >= start_season) & (seeds["Season"] <= end_season)]
        
    # generate pairwise match-ups 
    pairs = seeds\
        .merge(seeds, how='left', on=['Season'], suffixes=['_A', '_B'])\
        .drop(['Seed_A', 'Seed_B'], axis=1)\
        .query("TeamID_A < TeamID_B")\
        .rename(columns={'TeamID_A': 'TeamA', 'TeamID_B': 'TeamB'})\
        .assign(Teams = lambda x: x['TeamA'].astype(str) + '_' + x['TeamB'].astype(str))\
        .assign(ID = lambda x: x['Season'].astype(str) + '_' + x['Teams'])
        
    return pairs
        
len(create_match_ups(seeds, seasons=(2015, 2019)))

11390

In [ ]:
def reformat_obs(tourney_results):
    """ Given tourney results, reformat and append dataframe with unique ID and label """
    
    data = tourney_results.assign(
        TeamA = lambda x: x[['WTeamID','LTeamID']].min(axis=1), 
        TeamB = lambda x: x[['WTeamID','LTeamID']].max(axis=1), 
        Teams = lambda x: x['TeamA'].astype(str) + '_' + x['TeamB'].astype(str),
        ID = lambda x: x['Season'].astype(str) + '_' + x['Teams'],
        Won = lambda x: np.where(x['WTeamID'] < x['LTeamID'], 1, 0),
        ScoreTeamA = lambda x: np.where(x['WTeamID'] == x['TeamA'], x['WScore'], x['LScore']),
        ScoreTeamB = lambda x: np.where(x['WTeamID'] == x['TeamB'], x['WScore'], x['LScore']),
        ScoreDiff = lambda x: x['ScoreTeamA'] - x['ScoreTeamB']
    )
    
    cols_to_return = ['ID', 'Season', 'TeamA', 'TeamB', 'Teams', 'Won', 
                      'ScoreTeamA', 'ScoreTeamB', 'ScoreDiff', 'DayNum', 'NumOT', 'WLoc']
    
    return data[cols_to_return] 

In [ ]:
data = reformat_obs(tourney_results)
data.info()
display(data.query("Teams == '1314_1437'")) # UNC-Villanova head-to-heads 
data['Won'].value_counts()

### Create Seed Features

Here we append three features to our dataset: seedings of the respective teams for that season (`SeedA` for the lower-ID team and `SeedB` for the higher-ID team) and the difference between the two (`SeedDiff`=SeedB-SeedA). We expect the latter to be one of the most predictive features of the game outcome. 

In [ ]:
def append_seed_features(data, seeds):
    """ Create seed features and append to main dataframe """
    seeds = seeds.copy() 
    seeds['Seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
    seeds = seeds.set_index(['Season', 'TeamID'])
    df = data.merge(seeds, how='left', left_on=['Season', 'TeamA'], right_index=True)\
             .merge(seeds, how='left', left_on=['Season', 'TeamB'], right_index=True, suffixes=('A', 'B'))
    df['SeedDiff'] = df['SeedB'] - df['SeedA'] 
    return df 

In [ ]:
data2 = append_seed_features(data, seeds)
data2.query("Teams == '1314_1437'")

Not surprisingly, the higher a team is seeded, the more likely it is to win. 

In [ ]:
clf_eda_numeric(data2, feature='SeedA')

Conversely, the higher a team's opponent is seeded, the less likely a team is to win. 

In [ ]:
clf_eda_numeric(data2, feature='SeedB')

Perhaps the relative seeding of the teams is the most telling: conventional wisdom assumes the higher-seeded team wins, and the greater the gap in seedings, the higher the probability of winning. Our EDA below appears mostly consistent with our intuition.

In [ ]:
clf_eda_numeric(data2, feature='SeedDiff')

### Create historical tourney head-to-head features 

In [ ]:
def append_tourney_head_to_head(data):
    """ Given data w/ tourney results, append head-to-head features """
    
    # generate head-to-head metrics 
    tourney_h2h = data\
        .merge(data, how='left', on='Teams', suffixes=('_1', '_2'))\
        .query("Season_1 > Season_2")\
        .groupby(['Teams', 'Season_1'])\
        .agg({'Won_2': ['size', np.sum]})\
        .reset_index()
    tourney_h2h.columns = ['Teams', 'Season', 'TourneyH2HTotal', 'TourneyH2HWins']
    tourney_h2h['TourneyH2HLosses'] = tourney_h2h['TourneyH2HTotal'] - tourney_h2h['TourneyH2HWins']
    tourney_h2h['TourneyH2HEdge'] = tourney_h2h['TourneyH2HWins'] - tourney_h2h['TourneyH2HLosses']
    
    # append to main dataframe 
    output = data.merge(tourney_h2h, how='left', on=['Season','Teams']).fillna(0)
    for col in ['TourneyH2HTotal', 'TourneyH2HWins', 'TourneyH2HLosses', 'TourneyH2HEdge']:
        output[col] = output[col].astype(int)
    
    return output

In [ ]:
data3 = append_tourney_head_to_head(data2)
data3.query("Teams == '1314_1437'").T

# Fit Model

### Helper Functions 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone 
from sklearn.metrics import log_loss, roc_auc_score, average_precision_score, balanced_accuracy_score, \
    precision_score, recall_score, f1_score, roc_curve, precision_recall_curve

In [ ]:
# here we define a train/test split function to use all data preceding each season to train a model 

def train_test_period_split(data, season_to_predict, season_to_start=1985): 
    """ Split data into train vs. test datasets. 
        - Test set comprises only the single season specified; 
        - Train set comprises all data between season_to_start through the season before the one to predict on 
    """
    
    train_data = data.query("Season >= @season_to_start & Season < @season_to_predict")
    test_data = data.query("Season == @season_to_predict")

    return train_data, test_data

In [ ]:
def iterative_train(model, seasons_to_predict, data, features, season_to_start=1985): 
    """ For each season to predict, instantiate the model and fit to a different train set using all prior seasons 
        (beginning from season_to_start). Returns a dictionary with {season:model} value pairs. 
    """
    models = {} 
    for season in seasons_to_predict:
        train_data, _ = train_test_period_split(data, season, season_to_start)
        X_train, y_train = train_data[features], train_data['Won']
        models[season] = clone(model)
        models[season].fit(X_train, y_train)
    
    return models 

def iterative_predict(models, seasons_to_predict, data, features, season_to_start=1985):
    """ Iterate over list of models and predict on each relevant season. 
        Returns a subset of data (of relevant seasons) with prediction appended as a new column. 
    """
    pred_dfs = []
    
    for season in seasons_to_predict: 
        _, test_data = train_test_period_split(data, season)
        df = test_data.copy() 
        df['Pred'] = models[season].predict_proba(df[features])[:,1]
        pred_dfs.append(df)
        
    return pd.concat(pred_dfs)

In [ ]:
def evaluate_predictions(y_true, y_score, cls_threshold=.5, plot=False):
    """ Given labels and predictions, return a dictionary with classifier evaluation metrics. 
        Optionally, plot ROC and Precision-Recall curves. 
    """
    
    y_pred = y_score > cls_threshold
    metrics = {} 
    metrics['LogLoss'] = log_loss(y_true, y_score)
    metrics['AuROC'] = roc_auc_score(y_true, y_score)
    metrics['AuPR'] = average_precision_score(y_true, y_score)
    metrics['BalancedAcc'] = balanced_accuracy_score(y_true, y_pred)
    metrics['Precision'] = precision_score(y_true, y_pred)
    metrics['Recall'] = recall_score(y_true, y_pred)
    metrics['F1'] = f1_score(y_true, y_pred)
    
    if plot: 
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14,5))
        color1 = list(plt.rcParams['axes.prop_cycle'])[0]['color']
        color2 = list(plt.rcParams['axes.prop_cycle'])[1]['color']
        
        # TODO: refactor curve plotting code 
        # plot roc curve 
        fpr, tpr, thresholds = roc_curve(y_true, y_score)
        axes[0].plot(fpr, tpr, color=color1, lw=2, 
                     label='ROC curve (AuROC = %0.3f)' % metrics['AuROC'])
        axes[0].plot([0, 1], [0, 1], color='grey', lw=2, linestyle='--', label='No Skill')
        axes[0].set_xlim([0.0, 1.0])
        axes[0].set_ylim([0.0, 1.05])
        axes[0].set_xlabel('False Positive Rate')
        axes[0].set_ylabel('True Positive Rate')
        axes[0].set_title('ROC Curve')
        axes[0].legend(loc="lower right")
        
        # plot precision-recall curve 
        precisions, recalls, thresholds = precision_recall_curve(y_true, y_score)
        axes[1].plot(recalls, precisions, color=color2, lw=2, 
                     label='Precision-Recall Curve (AuPR = %0.3f)' % metrics['AuPR'])
        no_skill = sum(y_true) / len(y_true)
        axes[1].plot([0, 1], [no_skill, no_skill], color='grey', lw=2, linestyle='--', label='No Skill')
        axes[1].set_xlim([0.0, 1.0])
        axes[1].set_ylim([0.0, 1.05])
        axes[1].set_xlabel('Recall')
        axes[1].set_ylabel('Precision')
        axes[1].set_title('Precision-Recall Curve')
        axes[1].legend(loc="lower right")        
        
    return metrics 

In [ ]:
seasons_to_predict = [2015,2016,2017,2018,2019]

### Baseline Model (1): Logistic Regression w/ SeedDiff Only

In [ ]:
lr1_features = ['SeedDiff']
lr1 = LogisticRegression()
lr1_models = iterative_train(lr1, seasons_to_predict, data3, lr1_features)
lr1_preds = iterative_predict(lr1_models, seasons_to_predict, data3, lr1_features)
evaluate_predictions(lr1_preds['Won'], lr1_preds['Pred'], cls_threshold=.5, plot=True)

In [ ]:
lr2_features = ['SeedDiff','TourneyH2HEdge']
lr2 = LogisticRegression()
lr2_models = iterative_train(lr2, seasons_to_predict, data3, lr2_features)
lr2_preds = iterative_predict(lr2_models, seasons_to_predict, data3, lr2_features)
evaluate_predictions(lr2_preds['Won'], lr2_preds['Pred'], cls_threshold=.5, plot=True)

# Output Predictions

Here we output the predictions in the format required by Kaggle. For every possible pair-wise matchup between teams that qualified for March Madness each season, we need to predict the probability that the lower-ID team wins. Our prediction output comprises a CSV with two columns: ID, Pred 

In [22]:
# check submission format for Round 1 
sample_pred = pd.read_csv('data/MSampleSubmissionStage1_2020.csv')
display(sample_pred.head())
print(len(sample_pred))

,ID,Pred
0,2015_1107_1112,0.5
1,2015_1107_1116,0.5
2,2015_1107_1124,0.5
3,2015_1107_1125,0.5
4,2015_1107_1129,0.5


11390


In [ ]:
new_data = sample_pred.copy()
new_data.drop('Pred', axis=1, inplace=True)
new_data[['Season', 'TeamA', 'TeamB']] = new_data['ID'].str.split('_',expand=True).astype(int)
new_data = append_seed_features(new_data, seeds)
new_data = append_tourney_head_to_head(new_data)
new_data.info()